<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=97eab2e0b5360e7f2879cc7b96ef91f0adafa1cb43b977e011b5d2231f923aeb
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-23 16:03:47
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.41 C
-------------------
Today PnL: -81.30 K (-0.57%)
Current PnL: -18.95 L (-12.75%)
CY Booked + Current PnL: -7.51 L (-5.05%)
-------------------
Total profit:  1.54 L
Total loss:  -20.49 L
-------------------
Total Booked + Current PnL: 19.47 L (16.0%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.13%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 84.82 L (60.3%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 8.87%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,61.0,H-MC,3.63,196400.0,31730.0,11607.0,-1.64,19.27,5.91,26.31,89.0,2.73,1.40,28.79,X40N,NTT,AC
77,TTKPRESTIG,770.00,97.44,41.0,M-SC,3.10,85065.0,-15712.0,15805.0,-1.13,-15.59,18.58,0.09,245.0,-0.99,0.61,11.76,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.23,110685.0,10181.0,18927.0,-1.00,10.13,17.10,28.96,79.0,0.54,0.79,47.54,MH,ATH,METALS
57,RELIANCE,1533.00,-13.29,48.0,H-LC,4.59,216809.0,6443.0,22331.0,-0.06,3.06,10.30,13.68,37.0,0.29,1.54,20.09,XY25,NTT,REFINERIES
84,VOLTAS,1530.00,-1.92,46.0,H-MC,2.84,205905.0,14163.0,23597.0,-0.25,7.39,11.46,19.69,99.0,0.60,1.46,14.69,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3840.54,59.0,M-SC,5.35,122862.0,-24506.0,76322.0,3.72,-16.63,62.12,35.16,236.0,-0.32,0.87,25.49,XY24,NTT,AUTO
21,COFFEEDAY,80.0,-41.65,52.0,L-SC,42.51,85459.0,-28090.0,68136.0,3.61,-24.74,79.73,35.27,268.0,-0.41,0.61,109.16,XR,NTT,HOTELS
39,INDUSINDBK,1800.0,250.84,49.0,L-MC,12.24,43049.0,-34158.0,59550.0,2.85,-44.24,138.33,32.89,258.0,-0.57,0.31,18.55,XR,NTT,BANKS
60,SAIL,228.0,49.96,71.0,M-MC,8.97,236431.0,11469.0,156186.0,1.34,5.10,66.06,74.53,192.0,0.07,1.68,39.00,XY24,BTT,STEEL
66,SIS,528.0,2081.35,38.0,H-SC,4.85,88026.0,-23006.0,46610.0,1.29,-20.72,52.95,21.26,156.0,-0.49,0.63,18.93,OX40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-170.45,53.0,M-SC,4.38,81012.0,-21198.0,114591.0,-4.35,-20.74,141.45,91.37,208.0,-0.18,0.58,31.51,XY25,NTT,FINANCE
8,AWL,485.00,-65.14,50.0,H-MC,3.80,248396.0,-53355.0,214788.0,-4.06,-17.68,86.47,53.50,116.0,-0.25,1.77,10.33,XY24,NTT,FMCG
45,KANSAINER,340.00,-67.79,52.0,H-SC,0.84,221589.0,-48078.0,84403.0,-2.68,-17.83,38.09,13.47,138.0,-0.57,1.58,12.08,XY24,NTT,PAINTS
37,INDIAMART,4850.92,-54.64,27.0,H-SC,11.00,126723.0,3387.0,130373.0,-2.38,2.75,102.88,108.45,119.0,0.03,0.90,25.71,AR,ATH,MISC
56,RELAXO,1176.00,-37.26,40.0,H-SC,2.09,85578.0,-59582.0,135513.0,-2.17,-41.05,158.35,52.31,136.0,-0.44,0.61,14.55,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.11,44.0,H-MC,2.08,135792.0,4.0,42747.0,-1.42,0.00,31.48,31.48,107.0,0.00,0.97,12.12,X40,ATH,INSURANCE
23,DABUR,735.00,-7.80,38.0,H-MC,6.89,249775.0,1707.0,106704.0,-1.35,0.69,42.72,43.70,102.0,0.02,1.78,15.23,XY24,BTT,FMCG
29,HAVELLS,2069.16,-1.08,51.0,H-MC,2.02,245279.0,-2452.0,77508.0,-0.57,-0.99,31.60,30.30,92.0,-0.03,1.74,12.17,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.55,52.0,H-MC,3.80,201120.0,300.0,68481.0,-0.93,0.15,34.05,34.25,80.0,0.00,1.43,5.35,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,93.70,51.0,H-SC,9.34,122067.0,-20412.0,39928.0,0.98,-14.33,32.71,13.70,163.0,-0.51,0.87,44.07,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,97.44,41.0,M-SC,3.10,85065.0,-15712.0,15805.0,-1.13,-15.59,18.58,0.09,245.0,-0.99,0.61,11.76,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,52.0,H-SC,0.84,221589.0,-48078.0,84403.0,-2.68,-17.83,38.09,13.47,138.0,-0.57,1.58,12.08,XY24,NTT,PAINTS
18,CERA,9475.0,-21.96,37.0,H-SC,2.11,142140.0,-33763.0,75789.0,-1.94,-19.19,53.32,23.89,149.0,-0.45,1.01,22.76,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-22.03,44.0,H-MC,7.08,105554.0,-26281.0,68505.0,-0.92,-19.93,64.90,32.03,98.0,-0.38,0.75,18.23,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1258.85,80.0,M-SC,8.22,184913.0,3770.0,41698.0,0.46,2.08,22.55,25.10,235.0,0.09,1.32,48.19,XY24,NTT,HEALTHCARE
60,SAIL,228.00,49.96,71.0,M-MC,8.97,236431.0,11469.0,156186.0,1.34,5.10,66.06,74.53,192.0,0.07,1.68,39.00,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,61.0,H-MC,3.63,196400.0,31730.0,11607.0,-1.64,19.27,5.91,26.31,89.0,2.73,1.40,28.79,X40N,NTT,AC
17,CAMS,5211.76,-1.84,57.0,H-SC,2.63,111611.0,9607.0,34320.0,-0.69,9.42,30.75,43.06,122.0,0.28,0.79,28.15,X40N,ATH,MISC
84,VOLTAS,1530.00,-1.92,46.0,H-MC,2.84,205905.0,14163.0,23597.0,-0.25,7.39,11.46,19.69,99.0,0.60,1.46,14.69,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.23,110685.0,10181.0,18927.0,-1.00,10.13,17.10,28.96,79.0,0.54,0.79,47.54,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-41.42,47.0,M-SC,1.11,99068.0,7570.0,71180.0,-0.62,8.27,71.85,86.07,223.0,0.11,0.70,43.86,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,69.0,H-MC,7.02,186759.0,6951.0,105463.0,0.75,3.87,56.47,62.52,88.0,0.07,1.33,39.27,XR,NTT,BANKS
86,WIPRO,420.00,-11.98,48.0,M-LC,5.68,154771.0,3826.0,105631.0,-0.28,2.53,68.25,72.51,53.0,0.04,1.10,8.71,XR,NTT,IT
37,INDIAMART,4850.92,-54.64,27.0,H-SC,11.00,126723.0,3387.0,130373.0,-2.38,2.75,102.88,108.45,119.0,0.03,0.90,25.71,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-18.84,35.0,H-LC,7.18,292523.0,-23319.0,135994.0,-1.17,-7.38,46.49,35.67,5.0,-0.17,2.08,5.45,X40N,ATH,FMCG
31,HINDUNILVR,2922.00,-10.18,38.0,H-LC,10.28,252220.0,-4843.0,39977.0,-1.94,-1.88,15.85,13.67,24.0,-0.12,1.79,18.14,XY25,NTT,FMCG
5,ASIANPAINT,3465.66,-14.63,39.0,H-LC,7.65,215327.0,-36441.0,89641.0,-1.44,-14.47,41.63,21.13,27.0,-0.41,1.53,15.82,X40,ATH,PAINTS
30,HCLTECH,1943.91,-1.51,40.0,H-LC,9.92,224479.0,-17445.0,80723.0,-0.70,-7.21,35.96,26.15,8.0,-0.22,1.60,6.08,X40,ATH,IT
42,ITC,452.00,-38.77,41.0,H-LC,2.05,197165.0,-2973.0,23857.0,-0.92,-1.49,12.10,10.44,4.0,-0.12,1.40,4.00,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-4.62,50.0,H-LC,1.48,236812.0,-9339.0,88544.0,0.05,-3.79,37.39,32.18,16.0,-0.11,1.68,30.88,X200,ATH,IT
42,ITC,452.0,-38.77,41.0,H-LC,2.05,197165.0,-2973.0,23857.0,-0.92,-1.49,12.10,10.44,4.0,-0.12,1.40,4.00,X40,NTT,FMCG
65,SIEMENS,4671.5,0.81,54.0,H-LC,3.33,162020.0,-24075.0,71548.0,-0.10,-12.94,44.16,25.51,15.0,-0.34,1.15,19.40,AR,ATH,ELECTRICAL
51,NESTLEIND,1377.0,-11.15,44.0,H-LC,4.55,273406.0,7980.0,48803.0,-1.67,3.01,17.85,21.40,11.0,0.16,1.94,11.02,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.29,48.0,H-LC,4.59,216809.0,6443.0,22331.0,-0.06,3.06,10.30,13.68,37.0,0.29,1.54,20.09,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287866.0,-58090.0,132937.0,-0.37,-16.79,46.18,21.64,1.0,-0.44,2.05,1.98,X40,ATH,IT
42,ITC,452.00,-38.77,41.0,H-LC,2.05,197165.0,-2973.0,23857.0,-0.92,-1.49,12.10,10.44,4.0,-0.12,1.40,4.00,X40,NTT,FMCG
80,UNITDSPR,1644.00,-11.95,56.0,H-LC,7.49,230686.0,-3458.0,52089.0,0.10,-1.48,22.58,20.77,86.0,-0.07,1.64,5.34,X40N,NTT,BREWERIES
83,VBL,671.64,-18.84,35.0,H-LC,7.18,292523.0,-23319.0,135994.0,-1.17,-7.38,46.49,35.67,5.0,-0.17,2.08,5.45,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.51,40.0,H-LC,9.92,224479.0,-17445.0,80723.0,-0.70,-7.21,35.96,26.15,8.0,-0.22,1.60,6.08,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.81,54.0,H-LC,3.33,162020.0,-24075.0,71548.0,-0.10,-12.94,44.16,25.51,15.0,-0.34,1.15,19.40,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,41.0,H-LC,2.05,197165.0,-2973.0,23857.0,-0.92,-1.49,12.10,10.44,4.0,-0.12,1.40,4.00,X40,NTT,FMCG
19,CIPLA,1795.00,-21.49,42.0,H-LC,5.31,209213.0,4713.0,36696.0,-0.98,2.30,17.54,20.25,10.0,0.13,1.49,11.24,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-14.63,39.0,H-LC,7.65,215327.0,-36441.0,89641.0,-1.44,-14.47,41.63,21.13,27.0,-0.41,1.53,15.82,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.29,48.0,H-LC,4.59,216809.0,6443.0,22331.0,-0.06,3.06,10.30,13.68,37.0,0.29,1.54,20.09,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287866.0,-58090.0,132937.0,-0.37,-16.79,46.18,21.64,1.0,-0.44,2.05,1.98,X40,ATH,IT
40,INFY,2275.00,-18.16,49.0,H-LC,8.12,318968.0,6080.0,165608.0,-0.13,1.94,51.92,54.87,3.0,0.04,2.27,8.72,X40,BTT,IT
42,ITC,452.00,-38.77,41.0,H-LC,2.05,197165.0,-2973.0,23857.0,-0.92,-1.49,12.10,10.44,4.0,-0.12,1.40,4.00,X40,NTT,FMCG
83,VBL,671.64,-18.84,35.0,H-LC,7.18,292523.0,-23319.0,135994.0,-1.17,-7.38,46.49,35.67,5.0,-0.17,2.08,5.45,X40N,ATH,FMCG
1,ABB,7934.00,-39.18,53.0,H-LC,9.04,254338.0,-7281.0,126482.0,-1.20,-2.78,49.73,45.57,7.0,-0.06,1.81,8.22,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6950.00,60.0,L-SC,5.25,77327.0,-16483.0,93102.0,-1.66,-17.57,120.40,81.67,269.0,-0.18,0.55,49.75,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.51,85459.0,-28090.0,68136.0,3.61,-24.74,79.73,35.27,268.0,-0.41,0.61,109.16,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.52,42.0,H-SC,12.05,87049.0,-13914.0,108446.0,-0.18,-13.78,124.58,93.63,148.0,-0.13,0.62,29.66,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.89,51.0,H-SC,12.68,94320.0,-3660.0,25259.0,-0.14,-3.74,26.78,22.05,152.0,-0.14,0.67,37.10,XR,ATH,FINANCE
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.23,110685.0,10181.0,18927.0,-1.00,10.13,17.10,28.96,79.0,0.54,0.79,47.54,MH,ATH,METALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1258.85,80.0,M-SC,8.22,184913.0,3770.0,41698.0,0.46,2.08,22.55,25.10,235.0,0.09,1.32,48.19,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3840.54,59.0,M-SC,5.35,122862.0,-24506.0,76322.0,3.72,-16.63,62.12,35.16,236.0,-0.32,0.87,25.49,XY24,NTT,AUTO
71,TANLA,1963.11,-15.81,70.0,H-SC,14.38,225354.0,-45407.0,375350.0,0.18,-16.77,166.56,121.86,133.0,-0.12,1.60,76.29,AR,ATH,IT
81,VAIBHAVGBL,521.00,60.88,61.0,H-SC,6.03,139398.0,-43377.0,154439.0,-1.41,-23.73,110.79,60.77,125.0,-0.28,0.99,24.85,XR,NTT,JEWELLERY
60,SAIL,228.00,49.96,71.0,M-MC,8.97,236431.0,11469.0,156186.0,1.34,5.10,66.06,74.53,192.0,0.07,1.68,39.00,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.21
1,25,43.60
2,50,74.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.04
LC    32.39
MC    23.56
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.37
X40      14.71
X40N     11.41
XR       11.30
XY25     10.00
AR        8.38
OX40N     7.83
X200      1.68
MH        1.65
X5K       1.43
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.38
H-LC    25.88
H-MC    20.59
M-SC    13.14
M-LC     5.47
M-MC     2.66
L-SC     1.52
L-LC     1.04
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.84,-5.41,40.20
IT,12.97,-15.31,78.18
FINANCE,10.03,-16.31,67.06
MISC,7.03,-16.22,80.83
BANKS,6.15,-15.41,76.12
PAINTS,5.61,-17.05,34.50
ELECTRICAL,5.49,-6.65,44.06
INSURANCE,3.71,-6.10,45.76
AC,3.56,3.18,13.03


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3109550.0,22
XR,1321247.0,14
AR,1151364.0,9
X40,780714.0,10
X40N,601515.0,9
OX40N,547901.0,10
XY25,431960.0,7
SR,248956.0,2
X5K,116898.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3663079.0,29
M-SC,1396997.0,17
H-MC,1212045.0,15
H-LC,1179256.0,15
M-LC,380454.0,4
M-MC,304185.0,2
L-SC,247475.0,3
L-MC,59550.0,1
L-LC,38447.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1229638.0      6
M-SC       XY24       794440.0      7
H-SC       AR         787947.0      5
           XR         773379.0      7
H-MC       XY24       559336.0      4
H-LC       X40        492766.0      5
M-MC       XY24       304185.0      2
H-SC       X40N       300515.0      4
           OX40N      258732.0      4
           SR         248956.0      2
H-LC       X40N       224779.0      3
H-MC       X40        215782.0      4
H-LC       AR         198030.0      2
H-MC       XY25       167811.0      2
M-SC       AR         165387.0      2
L-SC       XR         161238.0      2
M-LC       XY24       157925.0      2
M-SC       OX40N      134427.0      4
M-LC       X5K        116898.0      1
M-SC       XR         115986.0      2
           XY25       114591.0      1
H-LC       XY25       111111.0      3
M-LC       XR         105631.0      1
H-MC       XR         105463.0      1
H-LC       X200        88544.0      1
L-SC       OX40N       86237.0      1
H-MC       X40N        76221.0      2
M-SC       X40         72166.0      1
H-MC       OX40N       68505.0      1
H-LC       XY24        64026.0      1
H-SC       MH          63912.0      1
L-MC       XR          59550.0      1
L-LC       XY25        38447.0      1
H-MC       MH          18927.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
